# Image Cluster Notebook

In [1]:
from ipyleaflet import basemaps
from ipyleaflet import TileLayer
import ipysheet
from IPython.display import Markdown
import ipywidgets
import leafmap
import numpy
import pandas
from pathlib import Path
import sys

import sys
sys.path.append('../../')
from ImageCluster.model.Clusterer import Clusterer
from ImageCluster.model.ImageHelper import ImageHelper


In [2]:
# ---
# The image listed below demonstrates how to add your image.  The image below
# is unavailable.
# ---
inFile = Path('/explore/nobackup/projects/ilab/data/ImageCluster/my4326.tif')
redBandId = 3
greenBandId = 2
blueBandId = 1

noDataValue = -9999.0 or None

# ---
# Output file will be written in the same directory as the input image,
# unless you change the outDirectory variable below.
# ---
outDirectory = inFile.parent  


In [3]:
# ----------------------------------------------------------------------------
# handleClick
# ----------------------------------------------------------------------------
def handleClick(change: dict) -> None:
    
    with output:
        
        if change.new == 'Next':

            nn = updateList(list(sl.options), list(sl.value))
            updateDict('N')
            sl.options = nn
            bt.value='Select:'

        if change.new == 'Done':
            updateDict('D')

        if change.new == 'Start Over':
        
            sl.options = opts
            updateDict('S')
            bt.value = 'Select:'

# ----------------------------------------------------------------------------
# relabel
# ----------------------------------------------------------------------------
def relabel(labelArray: numpy.ndarray, lookup: dict) -> numpy.ndarray:
    
    newLab = labelArray.copy()
    
    for k, v in lookup.items():

        if len(v)==1 & k==v[0]:
            pass
        else:
            newLab = numpy.where(numpy.isin(newLab, v), k, newLab)
    
    return newLab
            
# ----------------------------------------------------------------------------
# updateDict
# ----------------------------------------------------------------------------
def updateDict(op: str) -> None:
    
    if op == 'N':
        
        key = list(sl.value)[0]
        table[key] = list(sl.value)
        # print('Re-grouping : ', table)
    
    if op == 'D':
    
        if len(sl.options) > 0:
            
            key = list(sl.options)[0]
            table[key] = list(sl.options)
        
        print('Final Groups : ', table)
    
    if op == 'S':
        
        table.clear()
    
# ----------------------------------------------------------------------------
# updateList
# ----------------------------------------------------------------------------
def updateList(old: list, out: list) -> list:
    return [ele for ele in old if ele not in out]


In [4]:
# ---
# Ingest the input image.
# ---
inHelper = ImageHelper()

inHelper.initFromFile(inputFile=inFile, 
                      noDataValue=noDataValue, 
                      redBandId=redBandId, 
                      greenBandId=greenBandId, 
                      blueBandId=blueBandId)

# ---
# Create the clusters, put them into a Geotiff, then ingest the image.
# ---
labels = Clusterer.getClusters(bands=inHelper.getRgbBands(), numClusters=30)
labelsFile = outDirectory / (inFile.stem + '-labels' + inFile.suffix)
labelsDs = Clusterer.labelsToGeotiff(inHelper._dataset, labelsFile, labels)
lHelper = ImageHelper()
lHelper.initFromDataset(labelsDs, noDataValue)

In [5]:
corners = inHelper.getCorners()

# ---
# The only reason used leafmap and not ipyleaflet directly is to get the
# inspector tool under the wrench button on leafmap's map.
# ---
m = leafmap.Map(fullscreen_control=False,
                layers_control=True,
                search_control=False,
                draw_control=False,
                measure_control=False,
                scale_control=False,
                toolbar_control=True,
                center=[corners[1], corners[0]])

m.fit_bounds([[corners[1], corners[0]], [corners[3], corners[2]]])

m.add_raster(str(inFile),
             band=inHelper.getRgbIndexList(),
             vmin=inHelper._minValue,
             vmax=inHelper._maxValue,
             nodata=inHelper._noDataValue,
             opacity=0.5,
             layer_name=inFile.name
            )

m.add_raster(str(labelsFile),
             vmin=lHelper._minValue,
             vmax=lHelper._maxValue,
             nodata=lHelper._noDataValue,
             opacity=0.5,
             layer_name=labelsFile.name,
             palette='viridis',
            )

m

Map(center=[66.346511472519, -140.3222662042015], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

## Update the labels.
Select multiple values by clicking the mouse or using the arrow keys while pressing shirt, control or command.

In [6]:
opts = list(numpy.unique(labels))

sl = ipywidgets.SelectMultiple( \
        options=opts, 
        layout=(ipywidgets.Layout(height='200px', width='150px')))

bt = ipywidgets.ToggleButtons(options=['Select:', 'Next', 'Done', 'Start Over'], 
                              value='Select:')

output = ipywidgets.Output()
display(sl, bt, output)
table = {}
bt.observe(handleClick, names='value')

SelectMultiple(layout=Layout(height='200px', width='150px'), options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

ToggleButtons(options=('Select:', 'Next', 'Done', 'Start Over'), value='Select:')

Output()

## Edit the groups.
Edit cluster IDs in each group.  When finished, proceed to the next cell.

In [7]:
strTab = {}

for item in table:
    strTab[item] = ', '.join(str(i) for i in table[item])

df = pandas.DataFrame(strTab.items(), columns=['Class', 'Cluster ID'])
sheet = ipysheet.from_dataframe(df)
sheet.column_width = [1, 5]
sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=2, row_start=0, squeeze_row=Fa…

In [8]:
editedDf = ipysheet.to_dataframe(sheet)
strClusters = editedDf.to_dict()['Cluster ID']

finalClusters = {}

for key in strClusters:
    
    strCluster = strClusters[key]
    finalClusters[int(key)] = [int(i) for i in strCluster.split(',') if i]

print(finalClusters)
newClusters = relabel(labels, finalClusters)


{0: [0, 1, 2, 3, 4, 5, 6, 7, 8], 1: [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 2: [27, 28, 29]}


## Review the updated map.

In [9]:
clusterMapFile = outDirectory / (inFile.stem + '-cluster-map' + inFile.suffix)

cmDataset = Clusterer.labelsToGeotiff(inHelper._dataset, 
                                      clusterMapFile, 
                                      newClusters)

cmHelper = ImageHelper()
cmHelper.initFromDataset(cmDataset, noDataValue)

m.add_raster(str(clusterMapFile),
             vmin=cmHelper._minValue,
             vmax=cmHelper._maxValue,
             nodata=cmHelper._noDataValue,
             opacity=0.5,
             layer_name=clusterMapFile.name,
             palette='viridis'
            )

display(m)

Markdown(f'<h3>The cluster map is at<br><br>{clusterMapFile}</h3>')

Map(bottom=16505.0, center=[66.9030773436267, -138.87666489579095], controls=(ZoomControl(options=['position',…

<h3>The cluster map is at<br><br>/explore/nobackup/projects/ilab/data/ImageCluster/my4326-cluster-map.tif</h3>